# Deutsch–Jozsa Algorithm Tutorial (Part II)

The **Deutsch–Jozsa algorithm** is one of the most famous algorithms in quantum computing. The problem it solves has little practical value, but the algorithm itself is one of the earliest examples of a quantum algorithm that is exponentially faster than any possible deterministic algorithm for the same problem. It is also relatively simple to explain and illustrates several very important concepts (such as quantum oracles). As such, Deutsch–Jozsa algorithm is part of almost every introductory course on quantum computing.

This tutorial consists of several notebooks:
1. [Part I](./DeutschJozsaAlgorithmTutorial_P1.ipynb) covers the problem statement and the classical algorithm for solving it.
2. Part II introduces single-qubit quantum oracles and the Deutsch algorithm for solving the problem for one-bit input functions.
3. [Part III](./DeutschJozsaAlgorithmTutorial_P3.ipynb) introduces multi-qubit quantum oracles and the Deutsch-Jozsa algorithm for solving the general case of the problem.
4. [Part IV](./AQ/DeutschJozsaAlgorithmTutorial_P4.ipynb) shows how to run the algorithms in the cloud using Azure Quantum.

# Part II. Single-qubit quantum oracles and Deutsch algorithm

## Single-qubit oracles: the definition
A quantum oracle is a "black box" operation which is used as input to another algorithm. This operation is implemented in a way which allows to perform calculations not only on individual inputs, but also on superpositions of inputs. 

> This is *not* the same as being able to evaluate the function on all inputs at once, since you will not be able to extract the evaluation results!

Oracles are often defined using a classical function.
Single-qubit oracles implement single-bit classical functions of the form $f : \{0, 1\} \to \{0, 1\}$.

The oracle has to act on quantum states instead of classical values. 
To enable this, integer input $x$ is represented as a qubit state $|x\rangle$.

The type of oracles used in this tutorial are called *phase oracles*. A phase oracle $U_f$ encodes the value of the classical function $f$ it implements in the phase of the qubit state as follows:

$$U_f |x \rangle = (-1)^{f(x)} |x \rangle$$

In our case $f$ can return only two values, 0 or 1, which result in no phase change or adding a relative phase $-1$, respectively.

The effect of such an oracle on any single basis state is not particularly interesting: it just adds a global phase which is not something you can observe. However, if you apply this oracle to a *superposition* of basis states, its effect becomes noticeable. 
Remember that quantum operations are linear: if you define the effect of an operation on the basis states, you'll be able to deduce its effect on superposition states (which are just linear combinations of the basis states) using its linearity. 

## Implementing single-bit oracles in Q&#x23;

There are only 4 single-bit functions, so we can see how to implement them all as Q# oracles.

> **Note:**
All code snippets before Exercise 3 are just examples. They don't need to be modified and are not covered by tests.

### $f(x) \equiv 0$

This is the easiest function to implement: if $f(x) \equiv 0$, $U_f |x\rangle \equiv (-1)^0 |x\rangle = |x\rangle$. 
This means that $U_f$ is an identity - a transformation which does absolutely nothing! 
This is very easy to express in Q#:

In [ ]:
operation PhaseOracle_Zero (x : Qubit) : Unit {
    // Do nothing...
}

### $f(x) \equiv 1$

The second constant function is slightly trickier: if $f(x) \equiv 1$, $U_f |x\rangle \equiv (-1)^1 |x\rangle = - |x\rangle$. 
Now $U_f$ is a negative identity, i.e., a transformation which applies a global phase of $-1$ to the state. 
A lot of algorithms just ignore the global phase accumulated in them, since it is not observable. 
However, if we want to be really meticulous, we can use Q# library operation [Microsoft.Quantum.Intrinsic.R](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.r) which performs a given rotation around the given axis. 
When called with `PauliI` axis, this operation applies a global phase; since it doesn't take the input into account, it can be applied to any qubit, for example, the first qubit of the input.

In [ ]:
// Open namespace where the library function PI() is defined
open Microsoft.Quantum.Math;

operation PhaseOracle_One (x : Qubit) : Unit {
    // Apply a global phase of -1
    R(PauliI, 2.0 * PI(), x);
}

### $f(x) = x$

$$U_f |x\rangle = (-1)^{f(x)} |x\rangle = (-1)^{x} |x\rangle$$

This means that we need to do nothing if the qubit is in the $|0\rangle$ state, and apply a phase of $-1$ if it is in the $|1\rangle$ state. This is exactly the effect of the [Z gate](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.z); as a reminder, 

$$Z = \begin{bmatrix} 1 & 0 \\ 0 & -1\end{bmatrix}: Z |0\rangle = |0\rangle, Z |1\rangle = -|1\rangle$$

In [ ]:
operation PhaseOracle_X (x : Qubit) : Unit {
    Z(x);
}

## <span style="color:blue">Exercise 3</span>: Implement a quantum oracle for $f(x) = 1 - x$ in Q&#x23;

You're ready to try and write some actual quantum code! Implement a quantum oracle that implements the last single-bit function, $f(x) = 1 - x$.

**Input:** A qubit $x$.

**Goal:** Apply a transformation $U_f$ to the qubit, defined as $U_f |x\rangle = (-1)^{1-x} |x\rangle$.

<br/>

<details>
    <summary><b>Need a hint? Click Here</b></summary>
We can represent the effect of the oracle as

$$U_f |x\rangle = (-1)^{1-x} |x\rangle = (-1) \cdot (-1)^x |x\rangle$$

Can you get this effect by combining some of the previous oracles implementations?
</details>

In [ ]:
%kata T3_PhaseOracle_OneMinusX

operation PhaseOracle_OneMinusX (x : Qubit) : Unit is Adj + Ctl {
    // ...
}

## Solving the problem for single-bit functions: Deutsch algorithm

Now let's return to the problem of figuring out whether the given function is constant or balanced for single-bit functions.
What can we do if we are given a quantum oracle $U_f$ implementing the function $f(x)$?

There are two possible inputs to the function, $|0\rangle$ and $|1\rangle$. Let's see what happens if we apply the oracle to their superposition:

$$U_f \left( \frac{1}{\sqrt2} \big( |0\rangle + |1\rangle \big) \right) 
= \frac{1}{\sqrt2} \big( U_f |0\rangle + U_f |1\rangle \big) 
= \frac{1}{\sqrt2} \big( (-1)^{f(0)} |0\rangle + (-1)^{f(1)} |1\rangle \big)$$.

* If $f(0) = f(1)$, the relative phases of the two basis states are the same, and the resulting state is $|+\rangle = \frac{1}{\sqrt2} \big( |0\rangle + |1\rangle \big)$ (up to a global phase). 
* If $f(0) \neq f(1)$, the relative phases of the two basis states differ by a factor of $-1$, and the resulting state is $|-\rangle = \frac{1}{\sqrt2} \big( |0\rangle - |1\rangle \big)$ (up to a global phase). 

Now, the states $|+\rangle$ and $|-\rangle$ can be distinguished using measurement: if you apply the H gate to each of them, you'll get $H|+\rangle = |0\rangle$ if $f(0) = f(1)$, or $H|-\rangle = |1\rangle$ if $f(0) \neq f(1)$. This means that one oracle call does not let you calculate both $f(0)$ and $f(1)$, but it allows you to figure out whether $f(0) = f(1)$!

This is a special case of the Deutsch–Jozsa algorithm, called the Deutsch algorithm. 
The algorithm is very straightforward:

1. Start with a qubit in the $|0\rangle$ state.
2. Apply the H gate to the qubit.
3. Apply the oracle.
4. Apply the H gate to the qubit again.
5. Measure the qubit: if it is in the $|0\rangle$ state, the function is constant, otherwise it is balanced.

Note that this algorithm requires only <span style="color:green">$1$</span> oracle call, and always produces the correct result.

We can follow the steps of the algorithm for the constant and the balanced scenarios using a neat visualization. Since Deutsch algorithm deals only with states with real amplitudes, we can map all states on the unit circle, and follow the state evolution through the steps.

1-2. Start with a qubit in the $|0\rangle$ state and apply the H gate to the qubit.
   <img width=60% src="./img/2-plus-state.png"/>

3. Apply the oracle.  
   Here, the difference between the two scenarios becomes noticeable. In the constant scenario, $|0\rangle$ and $|1\rangle$ states get the same phase (either 1 or -1), so the state remains the same or acquires a global phase of -1, which is physically the same state. In the variable scenario, zero and one states get different phases, so the state changes!
   <img width=60% src="./img/3-apply-oracle.png"/>

4. Apply the H gate to the qubit again.
   Now, we get the $|0\rangle$ state for both constant scenarios and the $|1\rangle$ state for both variable scenarios!
   <img width=60% src="./img/4-apply-hadamard.png"/>

## <span style="color:blue">Exercise 4</span>: Implement the Deutsch algorithm!

**Input:** The "black box" oracle the implements $f(x)$.  

**Goal:** Return `true` if the function is constant ($f(0) = f(1)$), or `false` if it is balanced ($f(0) \neq f(1)$).
You can use only one oracle call!

> Useful documentation: [Q# conditional expressions](https://docs.microsoft.com/azure/quantum/user-guide/language/expressions/conditionalexpressions).

In [ ]:
%kata T4_DeutschAlgorithm

operation DeutschAlgorithm (oracle : (Qubit => Unit)) : Bool {
    // Allocate a qubit for the input.
    use q = Qubit();
    // Newly allocated qubits start in the |0⟩ state.

    // The first step is to prepare the qubit in the required state before calling the oracle.
    // A qubit can be transformed from the |0⟩ state to the |+⟩ state by applying a Hadamard gate H.
    // ...

    // Apply the oracle to the input qubit.
    // The syntax is the same as for applying any function or operation.
    // ...

    // Apply a Hadamard gate to the qubit again.
    // ...

    // Measure the qubit in the computational basis using the M operation,
    // and compare the result with Zero or One constants to get the Boolean return value.
    return ...;
}

## Running the algorithm

You have implemented the quantum version of the Deutsch algorithm - congratulations! The last step is to combine everything you've seen so far - run your code to check whether the oracles you've seen earlier in this tutorial implement constant or balanced functions.

> This is an open-ended task, and is not covered by a unit test. To run the code, execute the cell with the definition of the `Run_DeutschAlgorithm` operation first; if it compiled successfully without any errors, you can run the operation by executing the next cell (`%simulate Run_DeutschAlgorithm`).

> Note that this task relies on your implementations of the previous tasks. If you are getting the "No variable with that name exists." error, you might have to execute previous code cells before retrying this task. Don't forget to execute Q# code cells that define oracles in part II!

In [ ]:
open Microsoft.Quantum.Diagnostics;

operation RunDeutschAlgorithm () : String {
    // You can use Fact function to check that the return value 
    // of DeutschAlgorithm operation matches the expected value.
    // Uncomment the next line to run it.
    
    // Fact(DeutschAlgorithm(PhaseOracle_Zero) == true, "f(x) = 0 not identified as constant");
    
    // Run the algorithm for the rest of the oracles
    // ...
    
    
    // If all tests pass, report success!
    return "Success!";
}

In [ ]:
%simulate RunDeutschAlgorithm

Continue to [part III](./DeutschJozsaAlgorithmTutorial_P3.ipynb) to learn about multi-qubit quantum oracles and Deutsch-Jozsa algorithm for solving the general case of the problem.